In [20]:
import numpy as np
X = {}
y = {}
X["train"] = np.load("./train_feature_save.npy",allow_pickle=True )
y["train"] = np.load("./train_label_save.npy")
X["dev"] = np.load("./test_feature_save.npy")
y["dev"] = np.load("./test_label_save.npy")
X["train"].shape, X["dev"].shape

((9632, 768), (1071, 768))

In [21]:
y["train"]=y["train"].squeeze()
y["dev"]=y["dev"].squeeze()
y["train"].shape, y["dev"].shape

((9632,), (1071,))

In [22]:
from sklearn.linear_model import LogisticRegression
l2 = 10
model = LogisticRegression(penalty='l2', C=1/l2)

model.fit(X["train"], y["train"])
pred = np.reshape(model.predict_proba(X["dev"])[:, 1], (model.predict_proba(X["dev"])[:, 1].shape[0], 1))
model.score(X["dev"], y["dev"])

0.8935574229691877

In [23]:
import sklearn.metrics as mc
predl = model.predict(X["dev"])
mc.f1_score(y["dev"], predl)

0.2597402597402597

In [24]:
fpr, tpr, thresholds = mc.roc_curve(y["dev"], pred)
mc.auc(fpr, tpr)

0.8810535863553097

In [25]:
# change threshold
threshold = 0.25
decisions = (pred >= threshold).astype(int)
print(mc.f1_score(np.reshape(y["dev"], (1071, 1)), decisions))
print(mc.classification_report(np.reshape(y["dev"], (1071, 1)), decisions))

0.5348837209302325
              precision    recall  f1-score   support

         0.0       0.95      0.93      0.94       953
         1.0       0.49      0.58      0.53       118

    accuracy                           0.89      1071
   macro avg       0.72      0.76      0.74      1071
weighted avg       0.90      0.89      0.89      1071



In [26]:
w = np.concatenate((model.coef_, model.intercept_[None, :]), axis=1)
F_train = np.concatenate([X["train"], np.ones((X["train"].shape[0], 1))], axis=1) # Concatenating one to calculate the gradient with respect to intercept
F_dev = np.concatenate([X["dev"], np.ones((X["dev"].shape[0], 1))], axis=1)

error_train = model.predict_proba(X["train"])[:, 1] - y["train"]
error_dev = model.predict_proba(X["dev"])[:, 1] - y["dev"]

gradient_train = F_train * error_train[:, None]
gradient_dev = F_dev * error_dev[:, None]
gradient_train.shape, gradient_dev.shape

((9632, 769), (1071, 769))

In [27]:
probs = model.predict_proba(X["train"])[:, 1]
hessian = F_train.T @ np.diag(probs * (1 - probs)) @ F_train / X["train"].shape[0]
np.all(np.linalg.eigvals(hessian) > 0)

True

In [28]:
#from scipy import sparse
probs = model.predict_proba(X["train"])[:, 1]
hessian = F_train.T @ np.diag(probs * (1 - probs)) @ F_train / X["train"].shape[0] + l2 * np.eye(F_train.shape[1]) / X["train"].shape[0]
inverse_hessian = np.linalg.inv(hessian)

In [29]:
np.all(np.linalg.eigvals(hessian) > 0)

True

In [30]:
eps = 1 / X["train"].shape[0]
delta_k = -eps * inverse_hessian @ gradient_train.T
pred = np.reshape(model.predict_proba(X["dev"])[:, 1], (model.predict_proba(X["dev"])[:, 1].shape[0], 1))
grad_f = F_dev * (pred * (1 - pred))
delta_pred = grad_f @ delta_k
delta_pred.shape

(1071, 9632)

In [31]:
np.save("old_predictions.npy", pred)

In [32]:
def Remove(k, scores, test_idx):
    #print("test_idx", test_idx)
    #print("old")
    #print(pred[test_idx])
    #print("k", k)
    
    if pred[test_idx] > 0.25:
        top_k_index = scores[test_idx].argsort()[-k:]
    else:
        top_k_index = scores[test_idx].argsort()[:k]

    X_k = np.delete(X["train"], top_k_index, axis=0)
    y_k = np.delete(y["train"], top_k_index, axis=0)

        
    prediction = -np.sum(scores[test_idx][top_k_index])
    
    
    #print("prediction", prediction)

    return X_k, y_k, prediction

In [33]:
def new_train(k, dev_index, scores):
    X_k, y_k, prediction = Remove(k, scores, dev_index)
    if y_k.shape[0] == np.sum(y_k) or np.sum(y_k) == 0: # data contains only one class: 1
        return None, None, None, None

    # Fit the model again
    model_k = LogisticRegression(penalty='l2', C=0.1)
    model_k.fit(X_k, y_k)

    # predictthe probaility with test point
    test_point = X["dev"][dev_index]
    test_point=np.reshape(test_point, (1,-1))
    new = model_k.predict_proba(test_point)[0][1]

    change = -(model.predict_proba(test_point)[0][1] - model_k.predict_proba(test_point)[0][1])
    #change = model_k.predict_proba(test_point)[0][1]-model.predict_proba(test_point)[0][1]
    flip = ((model.predict(test_point)>0.25) == (model_k.predict(test_point)>0.25))
    
    
    #print("change    ", change)
    #print("old       ", model.predict_proba(test_point)[0][1])
    #print()
    
    
    
  
    return change, flip, prediction, new

# exact_k

In [34]:
def exact_k(test_idx):
    for k in range(1, np.sum(y["train"])):
        _, flip,_ = new_train(k, test_idx, delta_pred)
        if flip == False:
            return k
    return None

In [35]:
"""
# Loop over all dev points:
import warnings
warnings.filterwarnings("ignore")

exact_ks = []
for test_idx in range(X["dev"].shape[0]):
    k =exact_k(test_idx)
    print(test_idx, k)
    exact_ks.append(k)

exact_ks=np.array(exact_ks)
np.save("exact_ks_IP.npy", exact_ks)
"""

'\n# Loop over all dev points:\nimport warnings\nwarnings.filterwarnings("ignore")\n\nexact_ks = []\nfor test_idx in range(X["dev"].shape[0]):\n    k =exact_k(test_idx)\n    print(test_idx, k)\n    exact_ks.append(k)\n\nexact_ks=np.array(exact_ks)\nnp.save("exact_ks_IP.npy", exact_ks)\n'

# Find approximate k by IF

In [36]:
def pred_change(k, scores, test_idx):
    X_k, y_k, prediction = Remove(k, delta_pred, test_idx)
    

In [37]:
def approximate_k(test_idx):
    old = pred[test_idx].item()
    for k in range(1, X["train"].shape[0]):
        _, _, change = Remove(k, delta_pred, test_idx)
        
        #new_train(k, test_idx, delta_pred)
        #print("cahnge", change)
        if change == None:
            return None
        
        if old > 0.25 and old + change < 0.25:
            print("prediction", change)
            return k
        elif old < 0.25 and old + change > 0.25:
            print("prediction", change)
            return k
        
    return None

In [38]:
# Loop over all dev points:
import warnings
warnings.filterwarnings("ignore")

appro_ks = []
new_predictions = []
for test_idx in range(X["dev"].shape[0]):
    appro_k = approximate_k(test_idx)
    if appro_k != None:
        _, _, _, new_prediction = new_train(appro_k, test_idx, delta_pred)
        print(test_idx, appro_k)
        print("old", pred[test_idx], "new", new_prediction)
        print()
        appro_ks.append(appro_k)
        new_predictions.append(new_prediction)
    else:
        appro_ks.append(None)
        new_predictions.append(None)



prediction 0.12161116868390248
0 64
old [0.12870375] new 0.2465756815890714

prediction 0.17190194535628828
3 301
old [0.0782902] new 0.9924007062661724

prediction 0.17262269911073874
4 421
old [0.07740151] new 0.9840086773697108

prediction 0.14398285992259247
5 257
old [0.10602201] new 0.7526383189003227

prediction 0.04933598791316301
6 24
old [0.20135129] new 0.21900515295262846

prediction -0.04333015503211627
8 5
old [0.29092613] new 0.3216703833220703

prediction 0.19812322769861646
11 2120
old [0.05187755] new 0.9999975307713955

prediction 0.030022805857915398
12 15
old [0.22092318] new 0.3656453433295159

prediction 0.15746318765892553
13 306
old [0.09272507] new 0.9116961570613672

prediction -0.36760736636660685
15 196
old [0.61724166] new 0.07875444192825065

prediction 0.10615397899218555
16 93
old [0.14434841] new 0.5157814335406795

prediction 0.20436979008865036
17 2726
old [0.04563932] new 0.9999999817038746

prediction 0.18267294392862918
18 532
old [0.06744122] new

KeyboardInterrupt: 

In [ ]:
appro_ks= np.array(appro_ks)
new_predictions=np.array(new_predictions)
np.save("appro_ks_IP.npy", appro_ks)
np.save("new_predictions.npy", new_predictions)
np.save("old_predictions.npy", pred)

# Difference of change and prediction when k increasing

In [ ]:
"""
row = len(list(range(0, X["dev"].shape[0])))
col = len(list(range(1, np.sum(y["train"]))))
changes = np.zeros((row, col))
prediction = np.zeros((row, col))

import warnings
warnings.filterwarnings("ignore")
for i in range(0, X["dev"].shape[0]):
    print(i)
    for j in range(1, np.sum(y["train"]):
        changes[i, j-1], _, prediction[i, j-1] = new_train(j, i, delta_pred)
        
   
np.save("pred_change_app.npy", prediction)
np.save("changes.npy", changes)

difference = prediction - changes
np.save("difference_k_appk.npy", difference)
"""

In [82]:
pwd

'/Users/eciel/Desktop/y3s2_sum/orf/summer/privacy_methods/BERT/Bert_SST/Bert_linear_emo'